In [38]:
import yfinance as yf
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [ ]:
data = yf.download('^NSEI', '2015-01-01', '2024-12-15')

# 3. Data Preparation
data = pd.DataFrame(data)
data.reset_index(inplace=True)

# Check for missing values
if data.isnull().values.any():
    data.fillna(method='ffill', inplace=True)  


data['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
data.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
data = data[['Close']]  # Keep only the 'Close' column

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['^NSEI']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [34]:
print(data)

Empty DataFrame
Columns: [Close]
Index: []


In [12]:
train_data = data[:-50]
test_data = data[-50:]

In [13]:
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_scaled = scaler.fit_transform(train_data)

# Scale the test data using the same scaler
test_data_scaled = scaler.transform(test_data)


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
base_days = 10
x_train, y_train = [], []
for i in range(base_days, train_data_scaled.shape[0]):
    x_train.append(train_data_scaled[i-base_days:i])
    y_train.append(train_data_scaled[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(50, activation='tanh', return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(60, activation='tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(80, activation='tanh', return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(120, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(units=1))

c:\Users\Aayush\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
model.fit(x_train, y_train, epochs=50, batch_size=32, callbacks=[early_stopping])

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - loss: 0.0432
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0033
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0024
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0028
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0025
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0023
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0020
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0021
Epoch 9/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0021
Epoch 10/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0023
Epoch 11/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0017
Epoch 12/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0019
Epoch 13/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0020
Epoch 14/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 0.0018
Epoch 15/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0019
Epo

In [ ]:
x_test, y_test = [], []
for i in range(base_days, test_data_scaled.shape[0]):
    x_test.append(test_data_scaled[i-base_days:i])
    y_test.append(test_data_scaled[i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Predicting
predictions = model.predict(x_test)
predictions = predictions.reshape(-1, 1)

# Inverse transform to get actual price values
predictions = scaler.inverse_transform(predictions)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 958ms/step


In [ ]:
future_days = 10
last_sequence = x_train[-1:]  # This takes the last sequence from your training data
future_predictions = []

for i in range(future_days):
    pred = model.predict(last_sequence, verbose=0)
    future_predictions.append(pred[0][0])
    
    # Update last_sequence
    new_sequence = last_sequence[0]
    new_sequence = np.roll(new_sequence, -1)
    new_sequence[-1] = pred[0][0]
    last_sequence = new_sequence.reshape(1, base_days, 1)

In [ ]:
future_predictions = np.array(future_predictions).reshape(-1, 1)
future_predictions = scaler.inverse_transform(future_predictions)

In [ ]:
future_predictions

array([[25214.314],
       [25286.76 ],
       [25336.559],
       [25350.463],
       [25303.78 ],
       [25225.582],
       [25135.543],
       [25033.875],
       [24914.2  ],
       [24802.438]], dtype=float32)

In [ ]:
model.save('Niftystockmodel.keras')